# DenseNet121 - 64x64

### Setup

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = "3"

In [2]:
import gc
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split

In [3]:
print("CUDA avilable:",
     True if tf.config.list_physical_devices("GPU") else False)

CUDA avilable: True


In [4]:
SEED = 1234
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [5]:
input_shape = 64

### Preprocessing

In [6]:
PATH = "C:/Users/shj03/Desktop/git_files/ANBW/dataset"
train = pd.read_csv(f"{PATH}/train.csv")
test = pd.read_csv(f"{PATH}/test.csv")
submission = pd.read_csv(f"{PATH}/submission.csv")

In [ ]:
x_train = train.drop(["id", "digit", "letter"], axis=1).values
x_train = x_train.reshape(-1, 28, 28, 1)
x_train = np.where((x_train<=20)&(x_train!=0) ,0.,x_train)
x_train = x_train/255
x_train = x_train.astype("float32")
x_train_resized = np.zeros([2048, input_shape, input_shape, 3], dtype=np.float32)

for idx, img in enumerate(x_train):
    converted = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    resized = cv2.resize(converted, (input_shape, input_shape), interpolation=cv2.INTER_CUBIC)
    del converted
    x_train_resized[idx] = resized
    del resized
    gc.collect()

x_train_resized.shape

In [ ]:
y = train["digit"]
y_train = np.zeros((len(y), len(y.unique())))

for idx, digit in enumerate(y):
    y_train[idx, digit] = 1

y_train.shape

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train_resized, y_train, test_size=0.2, random_state=SEED, shuffle=True)

### Initialing model

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
BATCH = 32
EPOCHS = 100
CALLBACKS = [EarlyStopping(monitor="val_loss", verbose=1, patience=20)]

In [ ]:
densenet = tf.keras.applications.densenet.DenseNet121(
    weights=None,
    input_shape=(64, 64, 3),
    classes=10,
)

In [ ]:
model = tf.keras.Sequential()
model.add(densenet)

model.compile(loss="categorical_crossentropy",
             optimizer="Adam",
             metrics=["accuracy"])

In [ ]:
model.summary()

### Train model

In [ ]:
history = model.fit(x_train, y_train, batch_size=BATCH, epochs=EPOCHS, callbacks=CALLBACKS, validation_data=(x_valid, y_valid))

### Predict

In [ ]:
x_test = test.drop(["id", "letter"], axis=1).values
x_test = x_test.reshape(-1, 28, 28, 1)
x_test = np.where((x_test<=20)&(x_test!=0), 0., x_test)
x_test = x_test/255
x_test = x_test.astype("float32")
x_test_resized = np.zeros([20480, input_shape, input_shape, 3], dtype=np.float32)

for idx, img in enumerate(x_test):
    converted = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    resized = cv2.resize(converted, (input_shape, input_shape), interpolation=cv2.INTER_CUBIC)
    del converted
    x_test_resized[idx] = resized
    del resized
    gc.collect()

x_test_resized.shape

In [ ]:
submission['digit'] = np.argmax(model.predict(x_test_resized), axis=1)
submission.head()

In [ ]:
submission.to_csv('DenseNet121-64x64.csv', index=False)